# Récupération des données pour l'étude départementale

## 1- Importation des librairies externes et classiques

In [1]:
# Importation des librairies externes
!pip install git+https://github.com/InseeFrLab/Py-Insee-Data@master --ignore-install PyYAML
!pip install python-Levenshtein
!pip install openpyxl

  Cloning https://github.com/InseeFrLab/Py-Insee-Data (to revision master) to /tmp/pip-req-build-tithsz1u
  Running command git clone --filter=blob:none -q https://github.com/InseeFrLab/Py-Insee-Data /tmp/pip-req-build-tithsz1u
  Resolved https://github.com/InseeFrLab/Py-Insee-Data to commit 52139da402bc33da6b8892d7d3208f1d2ddfaa02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached pandas-1.3.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached pathlib2-2.3.6-py2.py3-none-any.whl (17 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached DateTime-4.3-py2.py3-none-any.whl (60 kB)
  Using cached re

In [2]:
# Importation des librairies 
import requests
import pandas as pd
import pynsee
import pynsee.download
from pynsee.sirene import *
import time
from tqdm import tqdm
from pynsee.utils.init_conn import init_conn
import os
from zipfile import ZipFile, Path
import io
import numpy as np

In [3]:
# token API insee
init_conn(insee_key=os.environ['INSEE_KEY'], insee_secret=os.environ['INSEE_SECRET'])

## 2- Téléchargement des données de SIRENE

In [4]:
### Téléchargement des données Sirene sur les établissements Siret de boucherie-charcuterie (code 4722Z) à l'instant T
# Téléchargement via Insee sirene data  by sirene.fr website

session = requests.session()
URL_ACCUEIL = "https://www.sirene.fr/sirene/public/creation-fichier"
URL_REQUETE_POST = "https://www.sirene.fr/sirene/public/verifier-creation-fichier.action"
URL_DEVIS="https://www.sirene.fr/sirene/client//sirene/client/devis-demande.action"

URL_RECAP="https://www.sirene.fr/sirene/client/telechargement-pret-fichier.action"
URL_TELECHARGEMENT_POST="https://www.sirene.fr/sirene/client/telechargement-fichier"
POST_DATA={
'fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.actif': 'true',
'__checkbox_fichier.caracteristiques.etat.ferme': 'true',
'fichier.caracteristiques.typeEtablissement': '1',
'fichier.caracteristiques.geoFile': 'false',
'fichier.caracteristiques.cjFile': 'false',
'fichier.caracteristiques.apeFile': 'false',
'fichier.caracteristiques.sirenSiretFile': 'false',
'fichiersCriteres.criteresGeographie.upload': '(binaire)',
'__multiselect_fichier.criteres.geographie.region.modalites': '',
'__multiselect_fichier.criteres.geographie.departement.modalites': '',
'fichiersCriteres.criteresActivite.upload': '(binaire)',
'__multiselect_fichier.criteres.activite.section.modalites': '',
'__multiselect_fichier.criteres.activite.division.modalites': '',
'__multiselect_fichier.criteres.activite.groupe.modalites': '',
'fichier.criteres.activite.sousClasse.modalites': '4722Z',
'__multiselect_fichier.criteres.activite.sousClasse.modalites': '',
'fichiersCriteres.criteresNature.upload': '(binaire)',
'__multiselect_fichier.criteres.nature.entrepriseNiveau1.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau2.modalites': '',
'__multiselect_fichier.criteres.nature.entrepriseNiveau3.modalites': '',
'__multiselect_fichier.criteres.trancheEffectif.modalites': '',
'fichier.criteres.dateCreation.date': '',
'fichier.criteres.dateCreation.dateFin': '',
'fichier.criteres.dateMaj.date': '',
'fichier.criteres.dateMaj.dateFin': '',
'fichiersCriteres.criteresSirenSiret.upload': '(binaire)',
'__multiselect_fichier.criteres.criteresCatEnt.modalites': '',
'strutsLocale': 'fr_FR',
}

result=session.get(URL_ACCUEIL)
result=session.post(URL_REQUETE_POST, data=POST_DATA)


result=session.get(URL_RECAP)

result=session.get(URL_DEVIS)

result=session.post(URL_TELECHARGEMENT_POST)

donnees_sirene_fr=pd.read_csv(io.StringIO(result.content.decode('utf-8')),dtype=str)


In [5]:
donnees_sirene_fr.shape
donnees_sirene_fr.sample(10)

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
17551,851773176,00017,85177317600017,O,2019-06-17,02,2019,4722ZB,2021-10-27T09:49:08,true,...,NaN,NaN,A,AU PETIT BOUCHER,NaN,NaN,NaN,47.22Z,NAFRev2,O
13381,811272053,00026,81127205300026,O,2016-09-01,00,2019,4722ZB,2021-10-27T09:17:34,false,...,NaN,NaN,A,BOUCHERIE TY BEDEC,NaN,NaN,NaN,47.22Z,NAFRev2,N
15033,825033418,00021,82503341800021,O,2018-03-22,NaN,NaN,NaN,2018-04-20T08:59:57,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
12039,795217025,00019,79521702500019,O,2013-09-01,02,2019,4722ZB,2021-10-27T09:03:58,true,...,NaN,NaN,A,LA CLAMARTOISE,NaN,NaN,LA CLAMARTOISE,47.22Z,NAFRev2,O
25,094129533,00017,09412953300017,O,1996-08-01,NaN,NaN,NaN,2011-07-23T13:00:14,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
667,325427649,00026,32542764900026,O,1982-12-04,NN,NaN,NaN,2008-01-04T19:06:47,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
18024,879481331,00020,87948133100020,O,2020-10-05,NaN,NaN,NaN,2020-10-15T10:15:04,false,...,NaN,NaN,A,BOUCHERIE DU PARC,NaN,NaN,BOUCHERIE DU PARC,47.22Z,NAFRev2,N
14639,822004321,00011,82200432100011,O,2016-08-11,NN,NaN,4722ZA,2019-06-24T14:06:25,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N
10265,533784203,00028,53378420300028,O,2011-11-21,11,2019,NaN,2021-10-27T08:41:47,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,O
3477,410775191,00027,41077519100027,O,2002-06-01,NN,NaN,NaN,2008-12-06T05:16:22,true,...,NaN,NaN,A,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,N


* les données de Sirene.fr sont des données d'entreprises France entière. Pour l'étude, le nombre d'établissement par département est considéré :

In [6]:
pd.set_option('display.max_columns', None)
donnees_sirene_fr.sample(10)

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,etatAdministratifUniteLegale,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,categorieJuridiqueUniteLegale,denominationUniteLegale,sigleUniteLegale,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,sexeUniteLegale,nomUniteLegale,nomUsageUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,prenomUsuelUniteLegale,pseudonymeUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,identifiantAssociationUniteLegale,economieSocialeSolidaireUniteLegale,caractereEmployeurUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,nicSiegeUniteLegale,dateDernierTraitementUniteLegale,categorieEntreprise,anneeCategorieEntreprise,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement
9107,518900071,00011,51890007100011,O,2009-12-16,01,2019,4722ZB,2021-11-30T08:44:26,true,A,O,NaN,2009-12-16,5499,BOUCHERIE VERNIER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,NaN,N,N,01,2019,00011,2021-11-30T08:44:26,PME,2019,NaN,1,NaN,RUE,VERNIER,06000,NICE,NaN,NaN,06088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,BOUCHERIE DE LA RIVIERA,NaN,NaN,NaN,47.22Z,NAFRev2,N
20127,975820978,00016,97582097800016,O,1958-01-01,00,2019,4722ZB,2021-12-11T03:42:56,true,A,O,NaN,1958-01-01,5499,GRANDE BOUCHERIE DE LA GARE ROUTIERE,NaN,LA BOUCHERIE BOURBONNAISE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,NaN,N,O,00,2019,00016,2021-12-11T03:42:56,PME,2019,NaN,30,NaN,PL,CHARLES DE GAULLE,03200,VICHY,NaN,NaN,03310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,LA BOUCHERIE BOURBONNAISE,NaN,NaN,NaN,47.22Z,NAFRev2,O
3660,414000042,00039,41400004200039,O,2021-07-01,NaN,NaN,4722ZC,2021-07-16T09:38:46,true,A,O,NaN,1997-09-16,1000,NaN,NaN,NaN,NaN,NaN,M,GOBERT,NaN,DIDIER,NaN,NaN,NaN,DIDIER,NaN,47.22Z,NAFRev2,NaN,NaN,N,NN,NaN,00039,2021-07-16T09:38:46,PME,2019,NaN,84,NaN,RUE,DU CHEMIN VERT,62200,BOULOGNE-SUR-MER,NaN,NaN,62160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,CHEVALINE DU PLATEAU,NaN,NaN,NaN,47.22Z,NAFRev2,N
19146,892575861,00018,89257586100018,O,2021-01-04,NaN,NaN,4722ZA,2021-01-06T03:35:12,true,A,O,NaN,2021-01-04,1000,NaN,NaN,NaN,NaN,NaN,M,D'ALMEIDA,NaN,ANTHONY,NaN,NaN,NaN,ANTHONY,NaN,47.22Z,NAFRev2,NaN,NaN,N,NaN,NaN,00018,2021-01-06T03:35:12,NaN,NaN,NaN,50,B,AV,DE ROMPSAY,17000,LA ROCHELLE,NaN,NaN,17300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,SOS BOUCHER 17,47.22Z,NAFRev2,N
6540,482087921,00011,48208792100011,O,2005-04-01,02,2019,4722ZB,2021-10-27T08:18:52,true,A,O,NaN,2005-04-01,5499,LA FERME DE MONTREUIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.22Z,NAFRev2,NaN,NaN

In [7]:
donnees_sirene_fr["departement"]=donnees_sirene_fr.codeCommuneEtablissement.str[:2]

donnees_sirene_fr["departement"]=np.where(donnees_sirene_fr.codeCommuneEtablissement.str[:2]=='97', donnees_sirene_fr.codeCommuneEtablissement.str[:3],donnees_sirene_fr.codeCommuneEtablissement.str[:2])
donnees_sirene_fr_agregees_par_departement = donnees_sirene_fr.groupby("departement").size().reset_index(name='NbBoucheriesParDep').set_index("departement")
donnees_sirene_fr_agregees_par_departement.describe()

,NbBoucheriesParDep
count,104.000000
mean,193.413462
std,168.100849
min,3.000000
25%,95.750000
50%,139.500000
75%,224.750000
max,876.000000


In [8]:
donnees_sirene_fr_agregees_par_departement

,NbBoucheriesParDep
departement,
01,214
02,120
03,105
04,79
05,70
...,...
974,102
975,3
976,8


## 3- Téléchargement des données du ministère de l'intérieur

In [ ]:
#Téléchargement des données de crimes et délits rapportés déclarés auprès des commissariats 
url_datagouv_min_interieur="https://www.data.gouv.fr/fr/datasets/r/d792092f-b1f7-4180-a367-d043200c1520"

donnees_police_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services PN 2018', skiprows=[1,2])
donnees_gendarmerie_nationale_2018=pd.read_excel(url_datagouv_min_interieur, sheet_name='Services GN 2018', skiprows=[1,2])

del donnees_police_nationale_2018[donnees_police_nationale_2018.columns[0]]
del donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[0]]

donnees_police_nationale_2018.set_index('Départements')
donnees_gendarmerie_nationale_2018.set_index('Départements')
#donnees_police_nationale_2018.head(10)
#donnees_gendarmerie_nationale_2018.head(10)



In [ ]:
donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[1:]]=donnees_gendarmerie_nationale_2018[donnees_gendarmerie_nationale_2018.columns[1:]].fillna(0).astype(int)
donnees_gendarmerie_nationale_2018

## Traitement des données de la Police Nationale

In [ ]:
donnees_police_nationale_2018_par_dep=donnees_police_nationale_2018.T
donnees_police_nationale_2018_par_dep.columns = donnees_police_nationale_2018_par_dep.iloc[0]
donnees_police_nationale_2018_par_dep=donnees_police_nationale_2018_par_dep.iloc[1: , :]

donnees_police_nationale_2018_par_dep.drop(donnees_police_nationale_2018_par_dep[donnees_police_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_police_nationale_2018_par_dep["departement"]=np.where(donnees_police_nationale_2018_par_dep.index.str[:2]=='97', donnees_police_nationale_2018_par_dep.index.str[:3],donnees_police_nationale_2018_par_dep.index.str[:2])

donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.groupby("departement").sum() 
donnees_police_nationale_2018_par_dep

* Agrégation par département, transposition et fusion :

In [ ]:
donnees_police_nationale_2018_par_dep = donnees_police_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_police_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Police Nationale"},inplace=True)

## Traitement des données de la Gendarmerie Nationale

In [ ]:
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018.T
donnees_gendarmerie_nationale_2018_par_dep.columns = donnees_gendarmerie_nationale_2018_par_dep.iloc[0]
donnees_gendarmerie_nationale_2018_par_dep=donnees_gendarmerie_nationale_2018_par_dep.iloc[1: , :]

donnees_gendarmerie_nationale_2018_par_dep.drop(donnees_gendarmerie_nationale_2018_par_dep[donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='98'].index,inplace=True)
donnees_gendarmerie_nationale_2018_par_dep["departement"]=np.where(donnees_gendarmerie_nationale_2018_par_dep.index.str[:2]=='97', donnees_gendarmerie_nationale_2018_par_dep.index.str[:3],donnees_gendarmerie_nationale_2018_par_dep.index.str[:2])

donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.groupby("departement").sum() 
donnees_gendarmerie_nationale_2018_par_dep

In [ ]:
donnees_gendarmerie_nationale_2018_par_dep = donnees_gendarmerie_nationale_2018_par_dep.sum(axis=1).reset_index("departement").set_index("departement")
donnees_gendarmerie_nationale_2018_par_dep.rename(columns={0: "Total des crimes et délits relevés par la Gendarmerie Nationale"},inplace=True)
donnees_gendarmerie_nationale_2018_par_dep

## Fusion des données de la Police Nationale et de la Gendarmerie Nationale 

In [ ]:
merged_crimes_delits_2018_par_dep = donnees_police_nationale_2018_par_dep.merge(donnees_gendarmerie_nationale_2018_par_dep, on="departement", how='left')
merged_crimes_delits_2018_par_dep

## 4 - Téléchargement des revenus localisés

In [ ]:
#Téléchargement des données Filosofi au niveau IRIS (millénisme 2018)
URL_FILO_DECL_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DEC_IRIS_2018.xlsx"
URL_FILO_DISP_2018="https://www.insee.fr/fr/statistiques/fichier/5055909/BASE_TD_FILO_DISP_IRIS_2018.xlsx"

donnees_filo_decl_2018=pd.read_excel(URL_FILO_DECL_2018, sheet_name='IRIS_DEC', skiprows=5)
donnees_filo_disp_2018=pd.read_excel(URL_FILO_DISP_2018, sheet_name='IRIS_DISP', skiprows=5)

In [ ]:
URL_FILO_ZIP_2018="https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_CSV_geo2021.zip"
#Télécharger le zip de l'URL
r=requests.get(URL_FILO_ZIP_2018)

open("base-cc-filosofi-2018_CSV_geo2021.zip", 'wb').write(r.content)
    
    
with ZipFile("base-cc-filosofi-2018_CSV_geo2021.zip",'r') as myzip:
    data = myzip.open("cc_filosofi_2018_COM-geo2021.CSV")
    data2 = myzip.open("cc_filosofi_2018_DEP-geo2021.CSV")
    
donnees_filo_communes_2018=pd.read_csv(data,sep=';')    
donnees_filo_departement_2018=pd.read_csv(data2,sep=';')    

In [ ]:
donnees_filo_communes_2018.head(10)
donnees_filo_departement_2018

In [ ]:
donnees_filo_departement_2018.describe()

In [ ]:
donnees_filo_departement_2018=donnees_filo_departement_2018.set_index("CODGEO")

In [ ]:
donnees_filo_departement_2018

## 5 - Téléchargement des populations légales

In [ ]:
#Téléchargement des populations légales du millénisme 2018 
URL_POP_LEG="https://www.insee.fr/fr/statistiques/fichier/4989724/ensemble.zip"

#Télécharger le zip de l'URL
r=requests.get(URL_POP_LEG)

open("ensemble.zip", 'wb').write(r.content)
    
    
with ZipFile("ensemble.zip",'r') as myzip:
    data = myzip.open("Departements.csv")

donnees_pop_leg_18=pd.read_csv(data,sep=';')     

In [ ]:
donnees_pop_leg_18.sample(10)

In [ ]:
donnees_pop_leg_18=donnees_pop_leg_18.set_index("CODDEP")

In [ ]:
donnees_pop_leg_18.describe()

## 6 - Téléchargement des données d'emplois localisées

In [ ]:
#Téléchargement des données d'emploi localisées
#donnees_emploi_statut_18 = pynsee.download.telechargerDonnees("ESTEL_T201", date = "2018")
#donnees_emploi_statut_18.sample(10)

#donnees_emploi_sexe_18 = pynsee.download.telechargerDonnees("ESTEL_T202", date = "2018")
#donnees_emploi_sexe_18.sample(10)


URL_TAUX_CHOMAGE_PAR_DEP="https://www.insee.fr/fr/statistiques/fichier/2012804/sl_etc_2021T2.xls"


donnees_chomage_par_departement_series_longues=pd.read_excel(URL_TAUX_CHOMAGE_PAR_DEP, sheet_name='Département', skiprows=3)

donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.iloc[:100]
donnees_chomage_par_departement_series_longues=donnees_chomage_par_departement_series_longues.set_index("Code")
donnees_chomage_par_departement_series_longues.index=donnees_chomage_par_departement_series_longues.index.astype(str)

In [ ]:
donnees_chomage_par_departement_series_longues.describe()

## Telechargement des données de répartition de gendarmes et policiers en 2019 sur le territoire (au niveau départemental)

In [ ]:
URL_NB_PN_GN_DEP = "https://www.insee.fr/fr/statistiques/fichier/5763601/IREF_SECUR21-F29.xlsx"
donnees_nb_pn_gn_dep=pd.read_excel(URL_NB_PN_GN_DEP, sheet_name='Figure 3', skiprows=1)
donnees_nb_pn_gn_dep = donnees_nb_pn_gn_dep.set_index("Unnamed: 0")
donnees_nb_pn_gn_dep=donnees_nb_pn_gn_dep.iloc[:100]
donnees_nb_pn_gn_dep=donnees_nb_pn_gn_dep.rename(columns={"Unnamed: 0": "CODGEO", "Unnamed: 1": "Département", "Taux pour 10 000 habitants":"Nb_PN_GN_dep_10k_hab"}, index={'Unnamed:0': 'CODDEP'})
donnees_nb_pn_gn_dep.index = donnees_nb_pn_gn_dep.index.astype(str)
donnees_nb_pn_gn_dep.index.names = ['CODDEP']
donnees_nb_pn_gn_dep['Nb_PN_GN_dep_100k_hab']=round(donnees_nb_pn_gn_dep['Nb_PN_GN_dep_10k_hab']*10)
donnees_nb_pn_gn_dep.drop(['Nb_PN_GN_dep_10k_hab'], axis = 1, inplace = True)
donnees_nb_pn_gn_dep


## 7 - Téléchargement du Code Officiel Géographique

* Pour des raisons de simplicité, nous travaillons en géographie 2021 :

In [ ]:
#cog_commune = pynsee.download.telechargerDonnees("COG_COMMUNE", date = "dernier")
#URL_COG_2021="https://www.insee.fr/fr/statistiques/fichier/5057840/commune2021-csv.zip"

#Télécharger le zip de l'URL
#r=requests.get(URL_COG_2021)

#open("commune2021-csv.zip", 'wb').write(r.content)
    
    
#with ZipFile("commune2021-csv.zip",'r') as myzip:
#    data = myzip.open("commune2021.csv")

#donnees_cog_2021=pd.read_csv(data)     

In [ ]:
#donnees_cog_2021

## 8 - Fusion des données

In [ ]:
donnees_2018=merged_crimes_delits_2018_par_dep.merge(donnees_filo_departement_2018,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_pop_leg_18,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_sirene_fr_agregees_par_departement,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_chomage_par_departement_series_longues,left_index=True,right_index=True)
donnees_2018=donnees_2018.merge(donnees_nb_pn_gn_dep,left_index=True,right_index=True)
donnees_2018=donnees_2018[['REG', 'Libellé', 'Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale', 'MED18' , 'TP6018', 'D118', 'D918', 'RD18', 'PTOT','T1_2018', 'NbBoucheriesParDep', 'Nb_PN_GN_dep_100k_hab']]
donnees_2018

* On se restreint au champ métropolitain car les données d'outre mer ne sont pas exhaustives sur toutes nos sources 

In [ ]:
donnees_2018 = donnees_2018[donnees_2018.index.str[:2]!='97']
donnees_2018
donnees_2018.describe()

* On fait l'hypothèse d'unicité du dépôt de plainte ou de signalement en gardarmerie ou bien en commissariat, ce qui permet de sommer les crimes et délits au niveau départemental

In [ ]:
donnees_2018['Total des crimes et délits relevés par la PN et GN'] = donnees_2018[['Total des crimes et délits relevés par la Police Nationale', 'Total des crimes et délits relevés par la Gendarmerie Nationale']].sum(axis=1)

In [ ]:
donnees_2018 = donnees_2018.rename(columns={"Total des crimes et délits relevés par la Police Nationale": "Crim_Del_PN", "Total des crimes et délits relevés par la Gendarmerie Nationale": "Crim_Del_GN", "Total des crimes et délits relevés par la PN et GN": "Crim_Del_PN_GN"})
donnees_2018

## 9 - On créé une base de données rapportant le nombre de crimes et délits, et le nombre de boucheries pour 100 000 habitants 


In [ ]:
donnees_2018_hab = donnees_2018
donnees_2018_hab['Nb_Boucherie_dep_hab']=round(donnees_2018_hab['NbBoucheriesParDep']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_GN_hab']=round(donnees_2018_hab['Crim_Del_GN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_PN_hab']=round(donnees_2018_hab['Crim_Del_PN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab['Crim_Del_PN_GN_hab']=round(donnees_2018_hab['Crim_Del_PN_GN']/donnees_2018_hab['PTOT']*100000)
donnees_2018_hab.drop(['Crim_Del_PN', 'Crim_Del_GN', 'PTOT', 'NbBoucheriesParDep'], axis=1, inplace = True)
donnees_2018_hab


In [ ]:
%store donnees_2018
%store donnees_2018_hab